In [214]:
from sklearn.model_selection import train_test_split
from pandas import DataFrame, read_csv, Series
from sklearn.metrics import confusion_matrix
from sklearn import preprocessing
from scipy.stats import norm
from __future__ import division
import pandas as pd 
import numpy as np

In [215]:
Location = r'C:\Users\ThinkYoga\Desktop\MLOT17\Spam\spambase.csv'
df = pd.read_csv(Location,header=None)
df.describe()

,0,1,2,3,4,5,6,7,8,9,...,48,49,50,51,52,53,54,55,56,57
count,4601.000000,4601.000000,4601.000000,4601.000000,4601.000000,4601.000000,4601.000000,4601.000000,4601.000000,4601.000000,...,4601.000000,4601.000000,4601.000000,4601.000000,4601.000000,4601.000000,4601.000000,4601.000000,4601.000000,4601.000000
mean,0.104553,0.213015,0.280656,0.065425,0.312223,0.095901,0.114208,0.105295,0.090067,0.239413,...,0.038575,0.139030,0.016976,0.269071,0.075811,0.044238,5.191515,52.172789,283.289285,0.394045
std,0.305358,1.290575,0.504143,1.395151,0.672513,0.273824,0.391441,0.401071,0.278616,0.644755,...,0.243471,0.270355,0.109394,0.815672,0.245882,0.429342,31.729449,194.891310,606.347851,0.488698
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,1.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.588000,6.000000,35.000000,0.000000
50%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.065000,0.000000,0.000000,0.000000,0.000000,2.276000,15.000000,95.000000,0.000000
75%,0.000000,0.000000,0.420000,0.000000,0.380000,0.000000,0.000000,0.000000,0.000000,0.160000,...,0.000000,0.188000,0.000000,0.315000,0.052000,0.000000,3.706000,43.000000,266.000000,1.000000
max,4.540000,14.280000,5.100000,42.810000,10.000000,5.880000,7.270000,11.110000,5.260000,18.180000,...,4.385000,9.752000,4.081000,32.478000,6.003000,19.829000,1102.500000,9989.000000,15841.000000,1.000000


In [216]:
# index for selecting data 0.75 is the percentage in training
index=np.array([1 if random.random() < 0.75 else 0 for i in range(len(df))])
X_train=np.array(df[df.columns[0:-1]])[index==1]
X_test=np.array(df[df.columns[0:-1]])[index==0]
Y_train=np.array(df[df.columns[-1]])[index==1]
Y_test=np.array(df[df.columns[-1]])[index==0]

In [217]:
# creamos los vectores de medias y desviaciones std para el set de spam y no-spam
m_nospam=np.mean(X_train[Y_train==0],0)
sd_nospam=np.std(X_train[Y_train==0],0) 
m_spam=np.mean(X_train[Y_train==1],0) 
sd_spam=np.std(X_train[Y_train==1],0) 

In [218]:
prob_spam_tot=sum(Y_train)/len(Y_train)
prob_nospam_tot=1-prob_spam_tot

In [224]:
#función clasificadora
#recibe una observación (un mail) y regresa 1 si es spam, 0 si en o.c.
def bayesClass(x):
    logspam = np.log(prob_spam_tot)
    lognospam = np.log(prob_nospam_tot)
    for i in range(len(x)): 
            val_pdf_sp = norm(m_spam[i],sd_spam[i]).pdf(x[i])
            val_pdf_nosp = norm(m_nospam[i],sd_nospam[i]).pdf(x[i])
            if(val_pdf_sp==0.0):
                logspam+=-len(x)*10.0
            else:
                logspam += np.log(val_pdf_sp)
            
            if(val_pdf_nosp==0.0):
                lognospam += -len(x)*10.0
            else:
                lognospam += np.log(val_pdf_nosp)         
    
    if logspam > lognospam:
        resp=1
    else: 
        resp=0
        
    return resp

In [225]:
predic = []
for i in range(len(X_test)):
    predic += [bayesClass(X_test[i])]
sum(predic)

617

In [226]:
print 'tasa de aciertos '+repr(sum(predic == Y_test)/len(Y_test))
print 'tasa de fracasos '+repr((len(Y_test)-sum(predic == Y_test))/len(Y_test))

tasa de aciertos 0.80678851174934729
tasa de fracasos 0.19321148825065274


In [227]:
#matriz de confusión de nuestra predicción
confusion_matrix(Y_test,predic)

array([[506, 196],
       [ 26, 421]], dtype=int64)

# NB SKlearn

In [228]:
from sklearn.naive_bayes import GaussianNB
clf = GaussianNB()
clf.fit(X_train, Y_train)

GaussianNB(priors=None)

In [229]:
pred = clf.predict(X_test)

In [230]:
sum(pred == Y_test) / len(Y_test)

0.80765883376849434

In [231]:
#matriz de confusión de sklearn
confusion_matrix(Y_test,pred)

array([[507, 195],
       [ 26, 421]], dtype=int64)